In [1]:
#import ipdb # alternative to pdb that works in jupyter notebook (pip3 install ipdb)
import os, subprocess, sys, re, time
from pathlib import Path
from pynq import Overlay
#from pynq import GPIO
from pynq import allocate
import math
import pandas as pd

from dma_receiver import DmaReceiver
from bram_loader import Bram_Loader
from continuous_monitoring_system_controller import ContinuousMonitoringSystemController
from riscv_instruction_decoder import get_riscv_instruction_name

BASE_DIR = Path('/home/xilinx/design_files')
PATH = BASE_DIR 

base = Overlay(str(BASE_DIR / 'imported_design.bit'))

ITEM_BYTE_SIZE = 8
FIFO_SIZE = 32768
# +4 because DMA seems to have it's own buffer it fills before dma.recvchannel.transfer is even called
buffer_length = min( base.continuous_monitoring_system_blocks.axi_dma_0.buffer_max_size // ITEM_BYTE_SIZE, FIFO_SIZE)# + 4) 

print('buffer_length =', buffer_length)
input_buffer = allocate(shape=(buffer_length,), dtype='u8')
input_buffer_2 = allocate(shape=(buffer_length,), dtype='u8')

dma_rec = base.continuous_monitoring_system_blocks.axi_dma_0.recvchannel

# https://pynq.readthedocs.io/en/v2.7.0/_modules/pynq/lib/axigpio.html
gpio_rst_n_out = base.axi_gpio_0.channel1[0]
gpio_rst_n_console_input = base.axi_gpio_0.channel1[1]
gpio_rst_n_console_output = base.axi_gpio_0.channel1[2]
#gpio_en_cpu_reset_server_request_put_out = base.axi_gpio_0.channel1[1]
gpio_pc_stream_m_axis_tlast_interval = base.axi_gpio_1.channel1

gpio_fifo_wr_count = base.axi_gpio_0.channel2[0:16]
gpio_fifo_rd_count = base.axi_gpio_0.channel2[16:32]

PERFORMANCE_EVENTS_COUNT = 37
PERFORMANCE_COUNTER_WIDTH = 7
PERFORMANCE_COUNTERS_OVERFLOW_MAP_WIDTH = PERFORMANCE_EVENTS_COUNT
PC_WIDTH = 64
INSTR_WIDTH = 32
CLK_COUNTER_WIDTH = 64
AXI_DATA_WIDTH = 512

def print_dma_channel_status(channel):
    print('dma.running =', channel.running)
    print('dma.idle =', channel.idle)
    print('dma.error =', channel.error)
    print('status =', hex(channel._mmio.read(channel._offset + 4)))
    
def reset_console_input(delay=0.001):
    gpio_rst_n_console_input.write(0)
    time.sleep(delay)
    gpio_rst_n_console_input.write(1)

def reset_console_output(delay=0.001):
    gpio_rst_n_console_output.write(0)
    time.sleep(delay)
    gpio_rst_n_console_output.write(1)
    
def reset_cpu(delay=0.001):
    ''' AXI GPIO controlled reset, active-low. '''
    #gpio_en_cpu_reset_server_request_put_out.write(0)
    reset_console_output()
    gpio_rst_n_out.write(0)
    time.sleep(delay)
    gpio_rst_n_out.write(1)
    time.sleep(delay)
    #gpio_en_cpu_reset_server_request_put_out.write(1)
    #time.sleep(delay)
    #gpio_en_cpu_reset_server_request_put_out.write(0)
    #time.sleep(delay)
    

def print_fifo_data_counts():
    print('gpio_fifo_wr_count =', gpio_fifo_wr_count.read())
    print('gpio_fifo_rd_count =', gpio_fifo_rd_count.read())
    
# def set_pc_stream_tlast_interval(items_count):
#     ''' Sets 32-bit value specifying how many items can 
#     arrive by a single dma.recvchannel.tranfer(ib) call. '''
#     gpio_pc_stream_m_axis_tlast_interval[0:32].write(items_count)

def console_send(s, end_byte=None):
    ''' Uses AXI GPIO and hardware FIFOs. '''
    console_input = base.console_io.axi_gpio_3.channel1[0:8]
    console_write_enable = base.console_io.axi_gpio_3.channel1[8] # using "edge_detector" to avoid continuous writing
    console_write_enable.off()
    for c in s:
        console_input.write(ord(c))
        console_write_enable.on()
        console_write_enable.off()
    if end_byte is not None:
        console_input.write(end_byte)
        console_write_enable.on()
        console_write_enable.off()
    
def console_data_available():
    console_output_empty = base.console_io.axi_gpio_3.channel2[8]
    return console_output_empty.read() == 0

def console_read():
    ''' Uses AXI GPIO and hardware FIFOs. '''
    s = ''
    console_output = base.console_io.axi_gpio_3.channel2[0:8]
    console_read_enable = base.console_io.axi_gpio_3.channel1[9] # using "edge_detector" to avoid continuous reading
    console_read_enable.off()
    while console_data_available():
        s += chr(console_output.read())
        console_read_enable.on()
        console_read_enable.off()
    return s

def instr_to_strings(instructions_integers):
    ''' Requires riscv-python-model installed.
    If network connection is available, "python3 -m pip install riscv-model.
    If not, then on separate machine with internet:
        python3 -m pip download riscv-model -d .  
    Then copy the downloaded .whl file to pynq and install with:
        python3 -m pip install <file.whl> -f ./ --no-index   
    Usage:
        instr_to_string([0xB60006F, 0xFE0791E3])
        '''
    instructions_string = ' 0x'.join(f'{ii:08X}' for ii in instructions_integers)
    return os.popen(f'riscv-machinsn-decode hexstring {instructions_string}').read().strip().split('\n')


####################################################################
# 

# def read_performance_event_names(f_name='performance_event_names.txt'):
#     ''' Reads events names from file, these were collected from CHERI-Flute source code by using this script:
#     https://github.com/michalmonday/Flute/blob/continuous_monitoring/builds/RV64ACDFIMSUxCHERI_Flute_verilator/vcd/read_vcd.py
#     '''
#     with open(f_name) as f:
#         return [line.strip() for line in f.readlines()]

def read_performance_event_names(f_name='performance_event_names_non_zero.csv'):
    ''' Reads events names from file, these were collected from CHERI-Flute source code by using this script:
    https://github.com/michalmonday/Flute/blob/continuous_monitoring/builds/RV64ACDFIMSUxCHERI_Flute_verilator/vcd/read_vcd.py
    '''
    with open(f_name) as f:
        return [line.strip().split(',')[2] for line in f.readlines()[1:]]


def pop_n_bits_value(val, n):
    ''' pop_n_bits_value(0xFFFF, 4) returns: (0xFFF, 0xF) '''
    bits_value = val & ((1<<n)-1)
    return val >> n, bits_value

def parse_fifo_item(fifo_item):
    ''' Parses a single fifo item (e.g. 1024 bits) numerical value. 
        Single fifo item = {59bits padding, performance_counters805(7bits*115counters), instr32, clk_counter_delta64, pc64}
        Padding is used because only power of 2s can be used as size in fifo generator block (or axi in general?)'''
    perf_counters = []
    for i in range(PERFORMANCE_EVENTS_COUNT):
        fifo_item, perf_counter = pop_n_bits_value(fifo_item, PERFORMANCE_COUNTER_WIDTH)
        perf_counters.append(perf_counter)
    fifo_item, perf_counters_overflow_map = pop_n_bits_value(fifo_item, PERFORMANCE_COUNTERS_OVERFLOW_MAP_WIDTH)
    fifo_item, pc = pop_n_bits_value(fifo_item, PC_WIDTH)
    fifo_item, clk_counter = pop_n_bits_value(fifo_item, CLK_COUNTER_WIDTH)
    fifo_item, instr = pop_n_bits_value(fifo_item, INSTR_WIDTH)
    return perf_counters, performance_counters_overflow_map, pc, clk_counter, instr

def get_dma_transfer(input_buffer, dma_rec, dont_wait=False):
    ''' Returns the number of transferred items, each having 1024 bits. '''
    dma_rec.transfer(input_buffer)
    if not dont_wait:
        dma_rec.wait() # depends on tlast
    items_transferred = math.floor(dma_rec.transferred * 64 / AXI_DATA_WIDTH / 8)
    print(f'items_transferred = {items_transferred}')
    return items_transferred

def parse_last_dma_transfer(input_buffer, items_transferred):
    pcs = []
    instrs = []
    instr_names = []
    clk_counters = []
    events = []
    events_overflows = []
    for i in range(items_transferred):
        chunks_per_item = math.ceil(AXI_DATA_WIDTH/64)
        start = chunks_per_item * i
        end = start + chunks_per_item
        fifo_item = int.from_bytes(bytes(input_buffer[start:end]), byteorder='little')
        perf_counters, perf_counters_overflow_map, pc, clk_counter, instr = parse_fifo_item(fifo_item)
        events.append(perf_counters)
        events_overflows.append(perf_counters_overflow_map)
        pcs.append(pc)
        clk_counters.append(clk_counter)
        instrs.append(instr)
        
        instr_names.append( get_riscv_instruction_name(instr) )
    instr_strings = instr_to_strings(instrs)
    return events, pcs, clk_counters, instrs, instr_names, instr_strings

def decode_riscv_instruction(instruction):
    opcode = instruction & 0x7F
    funct3 = (instruction >> 12) & 0x7
    funct7 = (instruction >> 25) & 0x7F
    return opcode, funct3, funct7

def get_riscv_instruction_name(instruction):
    opcode, funct3, funct7 = decode_riscv_instruction(instruction)
    if opcode == 0x33:
        if funct3 == 0x0:
            if funct7 == 0x0:
                return 'ADD'
            elif funct7 == 0x20:
                return 'SUB'
        elif funct3 == 0x1:
            return 'SLL'
        elif funct3 == 0x2:
            return 'SLT'
        elif funct3 == 0x3:
            return 'SLTU'
        elif funct3 == 0x4:
            return 'XOR'
        elif funct3 == 0x5:
            if funct7 == 0x0:
                return 'SRL'
            elif funct7 == 0x20:
                return 'SRA'
        elif funct3 == 0x6:
            return 'OR'
        elif funct3 == 0x7:
            return 'AND'
    elif opcode == 0x13:
        if funct3 == 0x0:
            return 'ADDI'
        elif funct3 == 0x1:
            return 'SLLI'
        elif funct3 == 0x2:
            return 'SLTI'
        elif funct3 == 0x3:
            return 'SLTIU'
        elif funct3 == 0x4:
            return 'XORI'
        elif funct3 == 0x5:
            if funct7 == 0x0:
                return 'SRLI'
            elif funct7 == 0x20:
                return 'SRAI'
        elif funct3 == 0x6:
            return 'ORI'
        elif funct3 == 0x7:
            return 'ANDI'
    elif opcode == 0x3:
        if funct3 == 0x0:
            return 'LB'
        elif funct3 == 0x1:
            return 'LH'
        elif funct3 == 0x2:
            return 'LW'
        elif funct3 == 0x4:
            return 'LBU'
        elif funct3 == 0x5:
            return 'LHU'
    elif opcode == 0x23:
        if funct3 == 0x0:
            return 'SB'
        elif funct3 == 0x1:
            return 'SH'
        elif funct3 == 0x2:
            return 'SW'
    elif opcode == 0x37:
        return 'LUI'
    elif opcode == 0x17:
        return 'AUIPC'
    elif opcode == 0x6F:
        return 'JAL'
    elif opcode == 0x67:
        return 'JALR'
    elif opcode == 0x63:
        if funct3 == 0x0:
            return 'BEQ'
        elif funct3 == 0x1:
            return 'BNE'
        elif funct3 == 0x4:
            return 'BLT'
        elif funct3 == 0x5:
            return 'BGE'
        elif funct3 == 0x6:
            return 'BLTU'
        elif funct3 == 0x7:
            return 'BGEU'
    elif opcode == 0x73:
        if funct3 == 0x0:
            if funct7 == 0x0:
                return 'ECALL'
            elif funct7 == 0x1:
                return 'EBREAK'
        elif funct3 == 0x1:
            if funct7 == 0x0:
                return 'CSRRW'
            elif funct7 == 0x1:
                return 'CSRRS'
            elif funct7 == 0x2:
                return 'CSRRC'
            elif funct7 == 0x5:
                return 'CSRRWI'
            elif funct7 == 0x6:
                return 'CSRRCI'
    return 'UNKNOWN'

event_names = read_performance_event_names()

print_dma_channel_status(dma_rec)
print_fifo_data_counts()

# set_pc_stream_tlast_interval(1000)


buffer_length = 32768
dma.running = True
dma.idle = False
dma.error = False
status = 0x0
gpio_fifo_wr_count = 0
gpio_fifo_rd_count = 0


In [2]:
def setup_cms(cms_ctrl):
    # Triggerring (exact address must match to start/stop trace)
    cms_ctrl.set_trigger_trace_start_address(0x1000)
    cms_ctrl.set_trigger_trace_end_address(0x80000106)  
    cms_ctrl.set_trigger_trace_start_address_enabled(False)
    cms_ctrl.set_trigger_trace_end_address_enabled(False)

    # Filtering (any address between lower bound and upper bound will be collected)
    cms_ctrl.set_monitored_address_range_lower_bound(0x0FFF)     #(0x80000000)
    cms_ctrl.set_monitored_address_range_upper_bound(0x800000FF)
    cms_ctrl.set_monitored_address_range_lower_bound_enabled(False)
    cms_ctrl.set_monitored_address_range_upper_bound_enabled(False)
    
    # Allow further trace collection if last traced program used "wfi"
    # (wait for interrupt) instruction which stops the trace.
    cms_ctrl.reset_wfi_wait()

# the long name is because of using hierarchy in Vivado block design
cms_ctrl_axi_gpio = base.continuous_monitoring_system_blocks.axi_gpio_to_cms_ctrl_interface.axi_gpio_cms_ctrl.channel1    
cms_ctrl = ContinuousMonitoringSystemController(cms_ctrl_axi_gpio)
setup_cms(cms_ctrl)

In [3]:
gpio_rst_n_out.write(0)

In [4]:
bram_loader = Bram_Loader(base.bram_loader.axi_gpio_2)
#bram_loader.load(PATH / 'riscv-example-baremetal-short.bin')
bram_loader.load(PATH / 'riscv-stack-mission.bin')

In [5]:
# program input needs to be available immediately, for that reason a separate reset line is used for processor
# and the console input fifo
reset_console_input()
console_send('AA', end_byte=ord('\n'))

In [6]:
print_fifo_data_counts()
reset_cpu()

gpio_fifo_wr_count = 0
gpio_fifo_rd_count = 0


In [7]:
print_fifo_data_counts()

gpio_fifo_wr_count = 1311
gpio_fifo_rd_count = 0


In [8]:
items_transferred = get_dma_transfer(input_buffer, dma_rec)#, dont_wait=True)

items_transferred = 1315


In [9]:
events, pcs, clk_counters, instrs, instr_names, instr_strings = parse_last_dma_transfer(input_buffer, items_transferred)

for i, (pc, instr, instr_str, clk_counter, instr_name) in enumerate(zip(pcs, instrs, instr_strings, clk_counters, instr_names)):
    print(f'{i:<4} CLK_DELTA={clk_counter:<14}PC={pc:>8X}    INSTR={instr:>08X}    INSTR_NAME={instr_name:<6}    {instr_str}')

0    CLK_DELTA=966872187     PC=    1010    INSTR=00028067    INSTR_NAME=JALR      jalr x0, x5, 0
1    CLK_DELTA=34            PC=80000000    INSTR=F14022F3    INSTR_NAME=UNKNOWN    Cannot decode f14022f3, invalid instruction
2    CLK_DELTA=1             PC=80000004    INSTR=00029C63    INSTR_NAME=BNE       bne x5, x0, .+24
3    CLK_DELTA=1             PC=80000008    INSTR=00001117    INSTR_NAME=AUIPC     auipc x2, 1
4    CLK_DELTA=1             PC=80000010    INSTR=4600006F    INSTR_NAME=JAL       jal x0, .+1120
5    CLK_DELTA=34            PC=80000470    INSTR=FF010113    INSTR_NAME=ADDI      addi x2, x2, -16
6    CLK_DELTA=58            PC=80000478    INSTR=DCBFF0EF    INSTR_NAME=JAL       jal x1, .-566
7    CLK_DELTA=34            PC=80000242    INSTR=F8010113    INSTR_NAME=ADDI      addi x2, x2, -128
8    CLK_DELTA=1             PC=80000244    INSTR=06113C23    INSTR_NAME=UNKNOWN    Cannot decode 06113c23, invalid instruction
9    CLK_DELTA=6             PC=8000024E    INSTR=01E00

182  CLK_DELTA=1             PC=800000B0    INSTR=FE843783    INSTR_NAME=UNKNOWN    Cannot decode fe843783, invalid instruction
183  CLK_DELTA=7             PC=800000C2    INSTR=F67FF0EF    INSTR_NAME=JAL       jal x1, .-154
184  CLK_DELTA=1             PC=80000028    INSTR=FE010113    INSTR_NAME=ADDI      addi x2, x2, -32
185  CLK_DELTA=20            PC=8000004E    INSTR=00008067    INSTR_NAME=JALR      jalr x0, x1, 0
186  CLK_DELTA=1             PC=800000C6    INSTR=FE843783    INSTR_NAME=UNKNOWN    Cannot decode fe843783, invalid instruction
187  CLK_DELTA=1             PC=800000CA    INSTR=0007C783    INSTR_NAME=LBU       lbu x15, 0(x15)
188  CLK_DELTA=1             PC=800000CE    INSTR=FE0791E3    INSTR_NAME=BNE       bne x15, x0, .-30
189  CLK_DELTA=1             PC=800000B0    INSTR=FE843783    INSTR_NAME=UNKNOWN    Cannot decode fe843783, invalid instruction
190  CLK_DELTA=7             PC=800000C2    INSTR=F67FF0EF    INSTR_NAME=JAL       jal x1, .-154
191  CLK_DELTA=1        

373  CLK_DELTA=1             PC=80000028    INSTR=FE010113    INSTR_NAME=ADDI      addi x2, x2, -32
374  CLK_DELTA=20            PC=8000004E    INSTR=00008067    INSTR_NAME=JALR      jalr x0, x1, 0
375  CLK_DELTA=1             PC=800000C6    INSTR=FE843783    INSTR_NAME=UNKNOWN    Cannot decode fe843783, invalid instruction
376  CLK_DELTA=1             PC=800000CA    INSTR=0007C783    INSTR_NAME=LBU       lbu x15, 0(x15)
377  CLK_DELTA=1             PC=800000CE    INSTR=FE0791E3    INSTR_NAME=BNE       bne x15, x0, .-30
378  CLK_DELTA=1             PC=800000B0    INSTR=FE843783    INSTR_NAME=UNKNOWN    Cannot decode fe843783, invalid instruction
379  CLK_DELTA=7             PC=800000C2    INSTR=F67FF0EF    INSTR_NAME=JAL       jal x1, .-154
380  CLK_DELTA=1             PC=80000028    INSTR=FE010113    INSTR_NAME=ADDI      addi x2, x2, -32
381  CLK_DELTA=20            PC=8000004E    INSTR=00008067    INSTR_NAME=JALR      jalr x0, x1, 0
382  CLK_DELTA=1             PC=800000C6    INSTR=F

579  CLK_DELTA=1             PC=800000B0    INSTR=FE843783    INSTR_NAME=UNKNOWN    Cannot decode fe843783, invalid instruction
580  CLK_DELTA=7             PC=800000C2    INSTR=F67FF0EF    INSTR_NAME=JAL       jal x1, .-154
581  CLK_DELTA=1             PC=80000028    INSTR=FE010113    INSTR_NAME=ADDI      addi x2, x2, -32
582  CLK_DELTA=20            PC=8000004E    INSTR=00008067    INSTR_NAME=JALR      jalr x0, x1, 0
583  CLK_DELTA=1             PC=800000C6    INSTR=FE843783    INSTR_NAME=UNKNOWN    Cannot decode fe843783, invalid instruction
584  CLK_DELTA=1             PC=800000CA    INSTR=0007C783    INSTR_NAME=LBU       lbu x15, 0(x15)
585  CLK_DELTA=1             PC=800000CE    INSTR=FE0791E3    INSTR_NAME=BNE       bne x15, x0, .-30
586  CLK_DELTA=1             PC=800000B0    INSTR=FE843783    INSTR_NAME=UNKNOWN    Cannot decode fe843783, invalid instruction
587  CLK_DELTA=7             PC=800000C2    INSTR=F67FF0EF    INSTR_NAME=JAL       jal x1, .-154
588  CLK_DELTA=1        

797  CLK_DELTA=1             PC=80000028    INSTR=FE010113    INSTR_NAME=ADDI      addi x2, x2, -32
798  CLK_DELTA=20            PC=8000004E    INSTR=00008067    INSTR_NAME=JALR      jalr x0, x1, 0
799  CLK_DELTA=1             PC=800000C6    INSTR=FE843783    INSTR_NAME=UNKNOWN    Cannot decode fe843783, invalid instruction
800  CLK_DELTA=1             PC=800000CA    INSTR=0007C783    INSTR_NAME=LBU       lbu x15, 0(x15)
801  CLK_DELTA=1             PC=800000CE    INSTR=FE0791E3    INSTR_NAME=BNE       bne x15, x0, .-30
802  CLK_DELTA=1             PC=800000B0    INSTR=FE843783    INSTR_NAME=UNKNOWN    Cannot decode fe843783, invalid instruction
803  CLK_DELTA=7             PC=800000C2    INSTR=F67FF0EF    INSTR_NAME=JAL       jal x1, .-154
804  CLK_DELTA=1             PC=80000028    INSTR=FE010113    INSTR_NAME=ADDI      addi x2, x2, -32
805  CLK_DELTA=20            PC=8000004E    INSTR=00008067    INSTR_NAME=JALR      jalr x0, x1, 0
806  CLK_DELTA=1             PC=800000C6    INSTR=F

958  CLK_DELTA=1             PC=80000028    INSTR=FE010113    INSTR_NAME=ADDI      addi x2, x2, -32
959  CLK_DELTA=20            PC=8000004E    INSTR=00008067    INSTR_NAME=JALR      jalr x0, x1, 0
960  CLK_DELTA=1             PC=800000C6    INSTR=FE843783    INSTR_NAME=UNKNOWN    Cannot decode fe843783, invalid instruction
961  CLK_DELTA=1             PC=800000CA    INSTR=0007C783    INSTR_NAME=LBU       lbu x15, 0(x15)
962  CLK_DELTA=1             PC=800000CE    INSTR=FE0791E3    INSTR_NAME=BNE       bne x15, x0, .-30
963  CLK_DELTA=1             PC=800000B0    INSTR=FE843783    INSTR_NAME=UNKNOWN    Cannot decode fe843783, invalid instruction
964  CLK_DELTA=7             PC=800000C2    INSTR=F67FF0EF    INSTR_NAME=JAL       jal x1, .-154
965  CLK_DELTA=1             PC=80000028    INSTR=FE010113    INSTR_NAME=ADDI      addi x2, x2, -32
966  CLK_DELTA=20            PC=8000004E    INSTR=00008067    INSTR_NAME=JALR      jalr x0, x1, 0
967  CLK_DELTA=1             PC=800000C6    INSTR=F

1137 CLK_DELTA=1             PC=80000230    INSTR=FE010113    INSTR_NAME=ADDI      addi x2, x2, -32
1138 CLK_DELTA=8             PC=80000240    INSTR=00008067    INSTR_NAME=JALR      jalr x0, x1, 0
1139 CLK_DELTA=1             PC=8000044A    INSTR=FEC42783    INSTR_NAME=LW        lw x15, -20(x8)
1140 CLK_DELTA=1             PC=8000044E    INSTR=0017879B    INSTR_NAME=UNKNOWN    Cannot decode 0017879b, invalid instruction
1141 CLK_DELTA=6             PC=8000045C    INSTR=FCE7FEE3    INSTR_NAME=BGEU      bgeu x15, x14, .-36
1142 CLK_DELTA=1             PC=80000438    INSTR=F8840713    INSTR_NAME=ADDI      addi x14, x8, -120
1143 CLK_DELTA=5             PC=80000446    INSTR=DEBFF0EF    INSTR_NAME=JAL       jal x1, .-534
1144 CLK_DELTA=1             PC=80000230    INSTR=FE010113    INSTR_NAME=ADDI      addi x2, x2, -32
1145 CLK_DELTA=8             PC=80000240    INSTR=00008067    INSTR_NAME=JALR      jalr x0, x1, 0
1146 CLK_DELTA=1             PC=8000044A    INSTR=FEC42783    INSTR_NAME=LW

In [10]:
df = pd.DataFrame(zip(pcs,clk_counters,instrs,instr_names,instr_strings), columns=['pc','clk_counter','instr', 'instr_names', 'instr_strings'])
df.iloc[:,0] = df.iloc[:,0].apply(lambda x: f'{x:08X}')
df[:20]

,pc,clk_counter,instr,instr_names,instr_strings
0,00001010,966872187,163943,JALR,"jalr x0, x5, 0"
1,80000000,34,4047512307,UNKNOWN,"Cannot decode f14022f3, invalid instruction"
2,80000004,1,171107,BNE,"bne x5, x0, .+24"
3,80000008,1,4375,AUIPC,"auipc x2, 1"
4,80000010,1,1174405231,JAL,"jal x0, .+1120"
5,80000470,34,4278255891,ADDI,"addi x2, x2, -16"
6,80000478,58,3703566575,JAL,"jal x1, .-566"
7,80000242,34,4160815379,ADDI,"addi x2, x2, -128"
8,80000244,1,101792803,UNKNOWN,"Cannot decode 06113c23, invalid instruction"
9,8000024E,6,31457391,JAL,"jal x0, .+30"


In [11]:
for instr in instrs: 
    name = get_riscv_instruction_name(instr)
    print(name)

JALR
UNKNOWN
BNE
AUIPC
JAL
ADDI
JAL
ADDI
UNKNOWN
JAL
LW
BGEU
ADDI
JAL
ADDI
JALR
LW
BGEU
ADDI
JAL
ADDI
JALR
LW
UNKNOWN
BGEU
ADDI
JAL
ADDI
JALR
LW
UNKNOWN
BGEU
ADDI
JAL
ADDI
JALR
LW
UNKNOWN
BGEU
ADDI
JAL
ADDI
JALR
LW
UNKNOWN
BGEU
ADDI
JAL
ADDI
JALR
LW
UNKNOWN
BGEU
ADDI
JAL
ADDI
JALR
LW
UNKNOWN
BGEU
ADDI
JAL
ADDI
JALR
LW
UNKNOWN
BGEU
ADDI
JAL
ADDI
JALR
LW
UNKNOWN
BGEU
ADDI
JAL
ADDI
JALR
LW
UNKNOWN
BGEU
ADDI
JAL
ADDI
JALR
LW
UNKNOWN
BGEU
ADDI
JAL
ADDI
JALR
LW
UNKNOWN
BGEU
AUIPC
JALR
JAL
ADDI
UNKNOWN
JAL
ADDI
JAL
UNKNOWN
BNE
UNKNOWN
JAL
ADDI
JALR
UNKNOWN
LBU
BNE
UNKNOWN
JAL
ADDI
JALR
UNKNOWN
LBU
BNE
UNKNOWN
JAL
ADDI
JALR
UNKNOWN
LBU
BNE
UNKNOWN
JAL
ADDI
JALR
UNKNOWN
LBU
BNE
UNKNOWN
JAL
ADDI
JALR
UNKNOWN
LBU
BNE
UNKNOWN
JAL
ADDI
JALR
UNKNOWN
LBU
BNE
UNKNOWN
JAL
ADDI
JALR
UNKNOWN
LBU
BNE
UNKNOWN
JAL
ADDI
JALR
UNKNOWN
LBU
BNE
UNKNOWN
JAL
ADDI
JALR
UNKNOWN
LBU
BNE
UNKNOWN
JAL
ADDI
JALR
UNKNOWN
LBU
BNE
UNKNOWN
JAL
ADDI
JALR
UNKNOWN
LBU
BNE
UNKNOWN
JAL
ADDI
JALR
UNKNOWN
LBU
BNE
UNKNOWN
JAL
ADDI
J

In [12]:
# print performance counters for the first 10 datapoints/instructions from data above
df = pd.DataFrame(events, columns=event_names)
df.iloc[:20]

,Core__BRANCH,Core__JAL,Core__JALR,Core__AUIPC,Core__LOAD,Core__STORE,Core__SERIAL_SHIFT,Core__LOAD_WAIT,Core__STORE_WAIT,Core__F_BUSY_NO_CONSUME,...,AXI4_Slave__AR_FLIT,AXI4_Slave__R_FLIT,AXI4_Slave__R_FLIT_FINAL,AXI4_Master__AW_FLIT,AXI4_Master__W_FLIT,AXI4_Master__W_FLIT_FINAL,AXI4_Master__B_FLIT,AXI4_Master__AR_FLIT,AXI4_Master__R_FLIT,AXI4_Master__R_FLIT_FINAL
0,0,0,0,0,1,0,0,20,0,0,...,1,8,1,0,0,0,0,1,8,1
1,0,0,1,0,0,0,0,0,0,27,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0,0,31,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,52,...,0,0,0,2,2,2,1,1,4,1
7,0,1,0,0,0,0,0,0,0,31,...,0,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
event_names

['Core__BRANCH',
 'Core__JAL',
 'Core__JALR',
 'Core__AUIPC',
 'Core__LOAD',
 'Core__STORE',
 'Core__SERIAL_SHIFT',
 'Core__LOAD_WAIT',
 'Core__STORE_WAIT',
 'Core__F_BUSY_NO_CONSUME',
 'Core__1_BUSY_NO_CONSUME',
 'Core__2_BUSY_NO_CONSUME',
 'L1I__LD',
 'L1I__LD_MISS',
 'L1I__LD_MISS_LAT',
 'L1I__TLB',
 'L1D__LD',
 'L1D__LD_MISS',
 'L1D__LD_MISS_LAT',
 'L1D__ST',
 'L1D__TLB',
 'TGC__READ',
 'TGC__READ_MISS',
 'AXI4_Slave__AW_FLIT',
 'AXI4_Slave__W_FLIT',
 'AXI4_Slave__W_FLIT_FINAL',
 'AXI4_Slave__B_FLIT',
 'AXI4_Slave__AR_FLIT',
 'AXI4_Slave__R_FLIT',
 'AXI4_Slave__R_FLIT_FINAL',
 'AXI4_Master__AW_FLIT',
 'AXI4_Master__W_FLIT',
 'AXI4_Master__W_FLIT_FINAL',
 'AXI4_Master__B_FLIT',
 'AXI4_Master__AR_FLIT',
 'AXI4_Master__R_FLIT',
 'AXI4_Master__R_FLIT_FINAL']

In [14]:
# event name format is "CATEGORY__NAME"
df.mean(axis=0).sort_values(ascending=False)[:-1]

L1I__LD                      2.247148
L1I__TLB                     2.123194
L1D__TLB                     1.010646
Core__1_BUSY_NO_CONSUME      0.664639
Core__2_BUSY_NO_CONSUME      0.636502
Core__F_BUSY_NO_CONSUME      0.606084
Core__STORE_WAIT             0.601521
Core__LOAD                   0.520152
AXI4_Slave__W_FLIT           0.498859
AXI4_Slave__AW_FLIT          0.498859
AXI4_Slave__W_FLIT_FINAL     0.498859
Core__STORE                  0.495817
L1D__LD                      0.488213
L1I__LD_MISS_LAT             0.479848
Core__LOAD_WAIT              0.353612
L1D__ST                      0.342966
AXI4_Master__B_FLIT          0.296578
AXI4_Slave__B_FLIT           0.286692
L1D__LD_MISS_LAT             0.280608
TGC__READ                    0.253992
AXI4_Master__W_FLIT          0.248669
AXI4_Master__AW_FLIT         0.248669
AXI4_Master__W_FLIT_FINAL    0.248669
Core__AUIPC                  0.110266
AXI4_Master__R_FLIT          0.062357
AXI4_Slave__R_FLIT           0.059316
Core__JAL   

In [15]:
def select_performance_counters(df):
    df_max = df.max(axis=0)
    return df_max[df_max != 0].index.tolist()

select_performance_counters(df)

['Core__BRANCH',
 'Core__JAL',
 'Core__JALR',
 'Core__AUIPC',
 'Core__LOAD',
 'Core__STORE',
 'Core__SERIAL_SHIFT',
 'Core__LOAD_WAIT',
 'Core__STORE_WAIT',
 'Core__F_BUSY_NO_CONSUME',
 'Core__1_BUSY_NO_CONSUME',
 'Core__2_BUSY_NO_CONSUME',
 'L1I__LD',
 'L1I__LD_MISS',
 'L1I__LD_MISS_LAT',
 'L1I__TLB',
 'L1D__LD',
 'L1D__LD_MISS',
 'L1D__LD_MISS_LAT',
 'L1D__ST',
 'L1D__TLB',
 'TGC__READ',
 'TGC__READ_MISS',
 'AXI4_Slave__AW_FLIT',
 'AXI4_Slave__W_FLIT',
 'AXI4_Slave__W_FLIT_FINAL',
 'AXI4_Slave__B_FLIT',
 'AXI4_Slave__AR_FLIT',
 'AXI4_Slave__R_FLIT',
 'AXI4_Slave__R_FLIT_FINAL',
 'AXI4_Master__AW_FLIT',
 'AXI4_Master__W_FLIT',
 'AXI4_Master__W_FLIT_FINAL',
 'AXI4_Master__B_FLIT',
 'AXI4_Master__AR_FLIT',
 'AXI4_Master__R_FLIT',
 'AXI4_Master__R_FLIT_FINAL']

In [16]:
df.max(axis=0).sort_values(ascending=False)[:-1]
df['Core__LOAD_WAIT'].sort_values()[-30:]

439       0
438       0
437       0
436       0
435       0
434       0
433       0
432       0
431       0
430       0
429       0
428       0
479       0
427       0
425       0
474       0
1020      7
1118      7
1072      7
0        20
1114     26
1016     26
1068     29
104      30
1219     30
557      34
11       46
15       46
989      55
108     102
Name: Core__LOAD_WAIT, dtype: int64

In [17]:
console_data_available()

True

In [18]:
console_read()

"Cookie monster is hungry, provide some cookies!\n'-' skips to the next character\nXX as two hex digits stores a single cookie\n> \nNo cookies??\n"

In [19]:
#console_send('AA', end_byte=ord('\n'))

In [20]:
#console_send('AAAA')

In [21]:
# import matplotlib.pyplot as plt
# plt.plot(pcs)
# plt.show()